# Module 14: Advanced Data Visualization for Research

**Estimated Time:** 45 minutes

## Learning Objectives

By the end of this module, you will be able to:

1. Apply principles of effective data visualization
2. Create publication-quality figures with matplotlib and seaborn
3. Choose appropriate chart types for different data
4. Visualize statistical results and uncertainty
5. Create interactive visualizations with Plotly
6. Avoid common visualization mistakes
7. Adapt visualizations for different audiences
8. Ensure accessibility in visualizations

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 11
plt.rcParams["font.family"] = "sans-serif"

# Create output directory
import os

os.makedirs("../notebooks/outputs/module_14", exist_ok=True)

# Set random seed
np.random.seed(42)

print("✓ Libraries imported successfully")
print("✓ Output directory created")

## 1. Principles of Effective Visualization

### Tufte's Principles

Edward Tufte's core principles:

1. **Maximize data-ink ratio**: Remove unnecessary elements
2. **Avoid chartjunk**: No 3D effects, unnecessary gridlines, decorations
3. **Show data variation, not design variation**: Focus on the data
4. **Use small multiples**: Repeated plots for comparison
5. **Integrate text and graphics**: Labels directly on plots

### Chart Selection Guide

In [ ]:
# Create chart selection guide
chart_guide = """CHART SELECTION GUIDE
========================================================================

COMPARING VALUES:
  Few categories (<7): Bar chart
  Many categories (>7): Horizontal bar chart or dot plot
  Over time: Line chart
  Part-to-whole: Stacked bar chart (avoid pie charts!)

DISTRIBUTION:
  Single variable: Histogram or density plot
  Multiple groups: Box plot, violin plot, or ridge plot
  Show individual points: Strip plot, swarm plot, or jittered scatter

RELATIONSHIPS:
  Two continuous variables: Scatter plot
  With third variable: Color/size/facets
  Correlation matrix: Heatmap
  Network data: Network diagram

STATISTICAL RESULTS:
  Group means with error: Bar + error bars or point + error bars
  Regression results: Coefficient plot (forest plot)
  Model fit: Residual plots, Q-Q plots
  Effect sizes: Forest plot with confidence intervals

AVOID:
  ✗ Pie charts (use bar chart instead)
  ✗ 3D charts (distorts perception)
  ✗ Dual y-axes (confusing)
  ✗ Too many colors (>7-8)
  ✗ Red-green for color blind audiences

========================================================================
"""

print(chart_guide)

with open("../notebooks/outputs/module_14/chart_selection_guide.txt", "w") as f:
    f.write(chart_guide)

print("\n✓ Chart selection guide saved")

## 2. Publication-Quality Figures

### Journal Requirements (Typical)

- **Resolution**: 300-600 DPI for print
- **Format**: TIFF, EPS, or PDF (vector when possible)
- **Color mode**: RGB for online, CMYK for print
- **Size**: Usually 3.5" (single column) or 7" (double column) width
- **Font**: 8-12 pt, readable when reduced
- **File size**: <10 MB typically

### Creating Publication-Quality Figures

In [ ]:
# Generate example data
np.random.seed(42)
groups = ["Control", "Treatment A", "Treatment B"]
data = {
    "Control": np.random.normal(100, 15, 50),
    "Treatment A": np.random.normal(115, 15, 50),
    "Treatment B": np.random.normal(125, 15, 50),
}

# BAD example: Default ugly plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: BAD
ax1.bar(groups, [data[g].mean() for g in groups])
ax1.set_title("BAD: Default Plot")
ax1.set_ylabel("Score")

# Right: GOOD - Publication quality
means = [data[g].mean() for g in groups]
sems = [stats.sem(data[g]) for g in groups]

x_pos = np.arange(len(groups))
bars = ax2.bar(
    x_pos,
    means,
    yerr=sems,
    capsize=5,
    color=["#377eb8", "#ff7f00", "#4daf4a"],
    edgecolor="black",
    linewidth=1.5,
    alpha=0.8,
)

ax2.set_ylabel("Cognitive Score (± SEM)", fontsize=12, fontweight="bold")
ax2.set_xlabel("Treatment Group", fontsize=12, fontweight="bold")
ax2.set_title("GOOD: Publication-Quality Plot", fontsize=14, fontweight="bold")
ax2.set_xticks(x_pos)
ax2.set_xticklabels(groups)
ax2.set_ylim([0, 150])
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)
ax2.grid(axis="y", alpha=0.3, linestyle="--")

# Add sample sizes
for i, (bar, group) in enumerate(zip(bars, groups)):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        5,
        f"n={len(data[group])}",
        ha="center",
        va="bottom",
        fontsize=9,
        color="black",
    )

# Add significance stars (example)
y_max = max(means) + max(sems) + 10
ax2.plot([0, 1], [y_max, y_max], "k-", linewidth=1.5)
ax2.text(0.5, y_max + 2, "**", ha="center", fontsize=14)

plt.tight_layout()
plt.savefig(
    "../notebooks/outputs/module_14/publication_quality_example.png",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
)
plt.show()

print("✓ Publication quality example saved")
print("\n📊 Key improvements in GOOD plot:")
print("   - Error bars (SEM)")
print("   - Color-blind friendly palette")
print("   - Sample sizes shown")
print("   - Clean, minimal design")
print("   - High resolution (300 DPI)")
print("   - Significance indicators")

## 3. Visualizing Statistical Results

### Forest Plot (Effect Sizes with CIs)

In [ ]:
# Create forest plot for effect sizes
studies = [
    "Smith et al. (2020)",
    "Jones et al. (2021)",
    "Lee et al. (2021)",
    "Garcia et al. (2022)",
    "Williams et al. (2023)",
    "META-ANALYSIS",
]
effect_sizes = [0.25, 0.45, 0.30, 0.55, 0.35, 0.38]
lower_ci = [0.10, 0.25, 0.15, 0.35, 0.20, 0.30]
upper_ci = [0.40, 0.65, 0.45, 0.75, 0.50, 0.46]
weights = [18, 22, 20, 24, 16, 100]

fig, ax = plt.subplots(figsize=(10, 7))

# Plot individual studies
for i, (study, es, low, high, weight) in enumerate(
    zip(studies[:-1], effect_sizes[:-1], lower_ci[:-1], upper_ci[:-1], weights[:-1])
):
    # Point estimate
    ax.scatter(
        es, i, s=weight * 10, color="#377eb8", edgecolor="black", linewidth=1.5, alpha=0.7, zorder=3
    )
    # Confidence interval
    ax.plot([low, high], [i, i], "k-", linewidth=2, alpha=0.7)
    ax.plot([low, low], [i - 0.1, i + 0.1], "k-", linewidth=2, alpha=0.7)
    ax.plot([high, high], [i - 0.1, i + 0.1], "k-", linewidth=2, alpha=0.7)

# Plot meta-analysis summary (diamond)
i = len(studies) - 1
meta_es, meta_low, meta_high = effect_sizes[-1], lower_ci[-1], upper_ci[-1]
diamond_x = [meta_es, meta_high, meta_es, meta_low, meta_es]
diamond_y = [i + 0.15, i, i - 0.15, i, i + 0.15]
ax.fill(diamond_x, diamond_y, color="#e74c3c", edgecolor="black", linewidth=2, alpha=0.8, zorder=4)

# Null effect line
ax.axvline(x=0, color="gray", linestyle="--", linewidth=2, alpha=0.7)

# Formatting
ax.set_yticks(range(len(studies)))
ax.set_yticklabels(studies, fontsize=11)
ax.set_xlabel("Effect Size (Cohen's d) with 95% CI", fontsize=12, fontweight="bold")
ax.set_title(
    "Meta-Analysis: Treatment Effect on Cognitive Outcomes", fontsize=14, fontweight="bold", pad=20
)
ax.set_xlim([-0.1, 0.9])
ax.grid(axis="x", alpha=0.3, linestyle="--")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

# Add effect size values
for i, (es, low, high) in enumerate(zip(effect_sizes, lower_ci, upper_ci)):
    ax.text(
        0.85,
        i,
        f"{es:.2f} [{low:.2f}, {high:.2f}]",
        ha="left",
        va="center",
        fontsize=10,
        fontweight="bold" if i == len(studies) - 1 else "normal",
    )

plt.tight_layout()
plt.savefig("../notebooks/outputs/module_14/forest_plot.png", dpi=300, bbox_inches="tight")
plt.show()

print("✓ Forest plot saved")

### Regression Diagnostics

In [ ]:
# Generate regression data
np.random.seed(42)
x = np.random.normal(50, 10, 100)
y = 2 * x + 10 + np.random.normal(0, 15, 100)

# Fit regression
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
y_pred = slope * x + intercept
residuals = y - y_pred

# Create 4-panel diagnostic plot
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Residuals vs Fitted
axes[0, 0].scatter(y_pred, residuals, alpha=0.6, edgecolor="black")
axes[0, 0].axhline(y=0, color="red", linestyle="--", linewidth=2)
axes[0, 0].set_xlabel("Fitted Values", fontsize=11, fontweight="bold")
axes[0, 0].set_ylabel("Residuals", fontsize=11, fontweight="bold")
axes[0, 0].set_title("Residuals vs Fitted", fontsize=12, fontweight="bold")
axes[0, 0].grid(alpha=0.3)

# 2. Q-Q Plot
stats.probplot(residuals, dist="norm", plot=axes[0, 1])
axes[0, 1].get_lines()[0].set_markerfacecolor("#377eb8")
axes[0, 1].get_lines()[0].set_markeredgecolor("black")
axes[0, 1].get_lines()[0].set_markersize(6)
axes[0, 1].get_lines()[0].set_alpha(0.6)
axes[0, 1].set_title("Normal Q-Q Plot", fontsize=12, fontweight="bold")
axes[0, 1].grid(alpha=0.3)

# 3. Scale-Location
standardized_residuals = residuals / np.std(residuals)
axes[1, 0].scatter(y_pred, np.sqrt(np.abs(standardized_residuals)), alpha=0.6, edgecolor="black")
axes[1, 0].set_xlabel("Fitted Values", fontsize=11, fontweight="bold")
axes[1, 0].set_ylabel("√|Standardized Residuals|", fontsize=11, fontweight="bold")
axes[1, 0].set_title("Scale-Location", fontsize=12, fontweight="bold")
axes[1, 0].grid(alpha=0.3)

# 4. Residuals Histogram
axes[1, 1].hist(residuals, bins=20, edgecolor="black", alpha=0.7, color="#377eb8")
axes[1, 1].axvline(x=0, color="red", linestyle="--", linewidth=2)
axes[1, 1].set_xlabel("Residuals", fontsize=11, fontweight="bold")
axes[1, 1].set_ylabel("Frequency", fontsize=11, fontweight="bold")
axes[1, 1].set_title("Distribution of Residuals", fontsize=12, fontweight="bold")
axes[1, 1].grid(alpha=0.3)

plt.suptitle("Regression Diagnostic Plots", fontsize=14, fontweight="bold", y=1.00)
plt.tight_layout()
plt.savefig(
    "../notebooks/outputs/module_14/regression_diagnostics.png", dpi=300, bbox_inches="tight"
)
plt.show()

print("✓ Regression diagnostic plots saved")
print("\n📈 What to look for:")
print("   1. Residuals vs Fitted: Should show random scatter (no pattern)")
print("   2. Q-Q Plot: Points should follow diagonal line (normality)")
print("   3. Scale-Location: Should show constant variance")
print("   4. Histogram: Should be roughly normal distribution")

## 4. Interactive Visualizations with Plotly

### When to Use Interactive vs. Static

**Use Interactive (Plotly) when:**
- Exploratory data analysis
- Presentations (can zoom, hover for details)
- Web dashboards
- Large datasets (can subset interactively)

**Use Static (matplotlib/seaborn) when:**
- Journal publications
- Print materials
- Exact control over appearance needed
- File size matters

In [ ]:
# Create interactive scatter plot with Plotly
np.random.seed(42)
n = 200
plot_data = pd.DataFrame(
    {
        "Age": np.random.normal(45, 15, n),
        "Cognitive_Score": np.random.normal(100, 15, n),
        "Group": np.random.choice(["Control", "Treatment"], n),
        "Education_Years": np.random.randint(8, 21, n),
    }
)

# Add correlation
plot_data.loc[plot_data["Group"] == "Treatment", "Cognitive_Score"] += 10
plot_data["Cognitive_Score"] += 0.5 * plot_data["Age"]

fig = px.scatter(
    plot_data,
    x="Age",
    y="Cognitive_Score",
    color="Group",
    size="Education_Years",
    hover_data=["Education_Years"],
    color_discrete_map={"Control": "#377eb8", "Treatment": "#ff7f00"},
    title="Interactive: Age vs Cognitive Score by Treatment Group",
    labels={"Cognitive_Score": "Cognitive Score", "Education_Years": "Years of Education"},
)

fig.update_traces(marker=dict(line=dict(width=1, color="black")))
fig.update_layout(
    font=dict(size=12), title_font_size=16, hovermode="closest", template="plotly_white"
)

# Save as HTML
fig.write_html("../notebooks/outputs/module_14/interactive_scatter.html")
fig.show()

print("✓ Interactive scatter plot saved as HTML")
print("\n💡 Hover over points to see details!")

## 5. Common Visualization Mistakes

### Mistake Gallery

In [ ]:
# Create examples of common mistakes
categories = ["A", "B", "C", "D"]
values = [30, 25, 35, 10]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Mistake 1: Truncated y-axis
axes[0, 0].bar(categories, values, color="#377eb8", edgecolor="black")
axes[0, 0].set_ylim([20, 40])  # BAD: Makes small differences look huge
axes[0, 0].set_title(
    "❌ MISTAKE 1: Truncated Y-Axis\n(Exaggerates differences)",
    fontsize=12,
    fontweight="bold",
    color="red",
)
axes[0, 0].set_ylabel("Value")

# Mistake 2: 3D pie chart
axes[0, 1].pie(
    values,
    labels=categories,
    autopct="%1.1f%%",
    colors=["#377eb8", "#ff7f00", "#4daf4a", "#e41a1c"],
)
axes[0, 1].set_title(
    "❌ MISTAKE 2: Pie Chart\n(Hard to compare, avoid!)",
    fontsize=12,
    fontweight="bold",
    color="red",
)

# Mistake 3: Too many colors
colors_bad = [
    "red",
    "orange",
    "yellow",
    "green",
    "blue",
    "indigo",
    "violet",
    "pink",
    "brown",
    "gray",
]
data_many = np.random.randint(10, 50, 10)
axes[1, 0].bar(range(10), data_many, color=colors_bad, edgecolor="black")
axes[1, 0].set_title(
    "❌ MISTAKE 3: Too Many Colors\n(Overwhelming, no meaning)",
    fontsize=12,
    fontweight="bold",
    color="red",
)
axes[1, 0].set_ylabel("Value")

# Mistake 4: Dual y-axes
x = np.arange(5)
y1 = np.array([10, 15, 13, 17, 20])
y2 = np.array([1000, 1500, 1300, 1700, 2000])
ax4 = axes[1, 1]
ax4_twin = ax4.twinx()
ax4.plot(x, y1, "b-o", label="Metric 1")
ax4_twin.plot(x, y2, "r-s", label="Metric 2")
ax4.set_ylabel("Metric 1", color="b")
ax4_twin.set_ylabel("Metric 2", color="r")
ax4.set_title(
    "❌ MISTAKE 4: Dual Y-Axes\n(Confusing, misleading)",
    fontsize=12,
    fontweight="bold",
    color="red",
)

plt.suptitle(
    "Common Visualization Mistakes to AVOID",
    fontsize=16,
    fontweight="bold",
    color="darkred",
    y=0.98,
)
plt.tight_layout()
plt.savefig("../notebooks/outputs/module_14/common_mistakes.png", dpi=300, bbox_inches="tight")
plt.show()

print("✓ Common mistakes example saved")
print("\n❌ Mistakes to avoid:")
print("   1. Truncated axes (unless clearly indicated)")
print("   2. Pie charts (use bar charts instead)")
print("   3. Too many colors (>7-8 is too many)")
print("   4. Dual y-axes (use facets or normalize instead)")

## 6. Accessibility in Visualizations

### Color Blindness Considerations

**~8% of men** and **~0.5% of women** have some form of color vision deficiency.

**Guidelines:**
- Avoid red-green combinations
- Use colorblind-friendly palettes
- Add patterns/textures when possible
- Include direct labels, not just color legend

**Colorblind-Friendly Palettes:**
- Seaborn: `colorblind` palette
- ColorBrewer: All palettes marked as "colorblind safe"
- Viridis, Plasma, Cividis (matplotlib)

In [ ]:
# Demonstrate colorblind-friendly palette
categories = ["Group A", "Group B", "Group C", "Group D"]
values = [30, 45, 35, 50]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# BAD: Red-green
colors_bad = ["red", "green", "darkred", "darkgreen"]
ax1.bar(categories, values, color=colors_bad, edgecolor="black", linewidth=1.5)
ax1.set_title(
    "❌ BAD: Red-Green (Not Colorblind Safe)", fontsize=12, fontweight="bold", color="red"
)
ax1.set_ylabel("Value")

# GOOD: Colorblind-friendly
colors_good = sns.color_palette("colorblind", n_colors=4)
bars = ax2.bar(categories, values, color=colors_good, edgecolor="black", linewidth=1.5)

# Add patterns for extra distinction
patterns = ["/", "\\", "|", "-"]
for bar, pattern in zip(bars, patterns):
    bar.set_hatch(pattern)

ax2.set_title(
    "✓ GOOD: Colorblind-Friendly + Patterns", fontsize=12, fontweight="bold", color="green"
)
ax2.set_ylabel("Value")

# Add value labels
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        height + 1,
        str(val),
        ha="center",
        va="bottom",
        fontsize=11,
        fontweight="bold",
    )

plt.suptitle("Color Accessibility Comparison", fontsize=14, fontweight="bold", y=1.00)
plt.tight_layout()
plt.savefig("../notebooks/outputs/module_14/colorblind_friendly.png", dpi=300, bbox_inches="tight")
plt.show()

print("✓ Colorblind-friendly example saved")
print("\n♿ Accessibility best practices:")
print("   - Use colorblind-safe palettes")
print("   - Add patterns/hatching for extra distinction")
print("   - Include direct labels on charts")
print("   - Ensure sufficient contrast")
print("   - Don't rely solely on color to convey information")

## 7. Exporting for Different Purposes

In [ ]:
# Create export guide
export_guide = """FIGURE EXPORT GUIDE
========================================================================

FOR JOURNAL PUBLICATION:
------------------------
Format: TIFF, EPS, or PDF (vector when possible)
Resolution: 300-600 DPI
Size: Check journal specifications (typically 3.5" or 7" width)
Color: RGB for online, CMYK for print
Font: 8-12 pt (readable when reduced)

Code:
plt.savefig('figure.pdf', dpi=300, bbox_inches='tight', 
            format='pdf', transparent=False)

FOR PRESENTATIONS (PowerPoint/Keynote):
---------------------------------------
Format: PNG or SVG
Resolution: 150-300 DPI (PNG) or vector (SVG)
Size: Fit slide dimensions (typically 10" x 7.5" for 4:3, or 13.33" x 7.5" for 16:9)
Font: Larger (14-18 pt minimum)

Code:
plt.savefig('figure.png', dpi=150, bbox_inches='tight',
            facecolor='white', edgecolor='none')

FOR WEB/SOCIAL MEDIA:
---------------------
Format: PNG or JPG
Resolution: 72-150 DPI
Size: Optimize for platform (Twitter: 1200x675, Instagram: 1080x1080)
File size: <1 MB

Code:
plt.savefig('figure.png', dpi=100, bbox_inches='tight',
            facecolor='white', optimize=True)

FOR POSTERS:
------------
Format: PDF or high-res PNG
Resolution: 150-300 DPI (posters viewed from distance)
Size: Actual size (e.g., 12" x 8" if that's poster space)
Font: Very large (18-24 pt minimum)

Code:
fig = plt.figure(figsize=(12, 8))  # Actual size
# ... create plot ...
plt.savefig('figure.pdf', dpi=200, bbox_inches='tight')

MATPLOTLIB SAVEFIG PARAMETERS:
------------------------------
fname: Filename (extension determines format)
dpi: Resolution (dots per inch)
facecolor: Background color
edgecolor: Border color
bbox_inches: 'tight' to minimize whitespace
transparent: True for transparent background
pad_inches: Padding around figure

COMMON FORMATS:
---------------
PDF: Vector, scalable, small file size, best for publications
EPS: Vector, older format, still required by some journals
PNG: Raster, supports transparency, good for web
TIFF: Raster, high quality, large files, required by some journals
SVG: Vector, great for web, editable in Illustrator
JPG: Raster, compressed, lossy, avoid for scientific figures

========================================================================
"""

print(export_guide)

with open("../notebooks/outputs/module_14/export_guide.txt", "w") as f:
    f.write(export_guide)

print("\n✓ Export guide saved to outputs/module_14/export_guide.txt")

## 8. Summary

### Key Takeaways

1. **Follow visualization principles**
   - Maximize data-ink ratio
   - Avoid chartjunk
   - Choose appropriate chart types

2. **Create publication-quality figures**
   - 300 DPI minimum
   - Clean, minimal design
   - Proper error bars
   - Sample sizes indicated

3. **Visualize statistical results clearly**
   - Forest plots for effect sizes
   - Diagnostic plots for assumptions
   - Show uncertainty (CIs, error bars)

4. **Use interactive when appropriate**
   - Plotly for exploration and presentations
   - Static for publications

5. **Avoid common mistakes**
   - No truncated axes (without clear indication)
   - No pie charts
   - Limit colors (<7-8)
   - No dual y-axes

6. **Ensure accessibility**
   - Colorblind-friendly palettes
   - Patterns/textures for distinction
   - Direct labels
   - Sufficient contrast

7. **Export properly for purpose**
   - PDF/TIFF for journals
   - PNG for presentations
   - Appropriate resolution
   - Check file size

### Visualization Checklist

Before finalizing any figure:

☐ Appropriate chart type for data and message
☐ Clear, descriptive title
☐ Axis labels with units
☐ Legible font size (8-12 pt for publication)
☐ Colorblind-friendly palette
☐ Error bars or uncertainty shown
☐ Sample sizes indicated
☐ Legend if multiple groups
☐ No unnecessary elements (chartjunk)
☐ Correct DPI for purpose
☐ Appropriate file format
☐ Proofread all text
☐ Checked by colleague

## Additional Resources

### Books
- Tufte, E. R. (2001). *The Visual Display of Quantitative Information*.
- Wilke, C. O. (2019). *Fundamentals of Data Visualization*. (Free online: https://clauswilke.com/dataviz/)
- Few, S. (2012). *Show Me the Numbers*.

### Online Resources
- Matplotlib Gallery: https://matplotlib.org/stable/gallery/
- Seaborn Gallery: https://seaborn.pydata.org/examples/
- Plotly Gallery: https://plotly.com/python/
- ColorBrewer (colorblind-safe): https://colorbrewer2.org
- Data Viz Project (chart selection): https://datavizproject.com

### Tools
- Color blindness simulator: https://www.color-blindness.com/coblis-color-blindness-simulator/
- Contrast checker: https://webaim.org/resources/contrastchecker/
- Color palette generator: https://coolors.co

## Next Steps

In **Module 15: Capstone Project**, you'll:
- Integrate all skills learned in this course
- Design and execute a complete research project
- From hypothesis to publication-ready manuscript
- Apply best practices in reproducibility, ethics, and communication

Put it all together in a comprehensive capstone! 🎓